In [1]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np

# Evaluation on multimodal test set

In [13]:
multimodal_preds = pd.read_csv('/home/robbe/IM2DeepMulti/preds/output/Test-BestPearsonBestModel-2024-04-08_09-33-13.csv')
decoy_unimodal_preds = pd.read_csv('/home/robbe/IM2DeepMulti/preds/decoyeval/DecoyModelUniSamples-2024-04-10_14-10-35.csv')
decoy_multirandom_preds = pd.read_csv('/home/robbe/IM2DeepMulti/preds/decoyeval/DecoyModelMultiRandom-2024-04-10_14-45-34.csv')

In [3]:
# Plot 3 overlapping scatter plots, twice, once for smallest target-prediction, once for largest target-prediction
import matplotlib.pyplot as plt
plt, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 10))



,Modified sequence,charge,CCS,between_brackets,seq,modifications,multimodal,tr,predicted_CCS1,predicted_CCS2
0,IAQPGDHVSVTGIFLPVLR,3,[514.00706906 558.5445672 ],NaN,IAQPGDHVSVTGIFLPVLR,Not modified,True,0,598.70856,558.99567
1,LGVHPLSCHGWVLGEHGDSSVPVWSGMNVAGVSLK,4,[931.94618833 802.74381079],NaN,LGVHPLSCHGWVLGEHGDSSVPVWSGMNVAGVSLK,8|Carbamidomethyl,True,0,902.07940,827.83356
2,AKPVVSFIAGITAPPGR,3,[485.41089318 554.37456912],NaN,AKPVVSFIAGITAPPGR,Not modified,True,0,548.78406,502.83514
3,LGYEEHIPGQTR,2,[419.07380157 406.42555945],NaN,LGYEEHIPGQTR,Not modified,True,0,415.85748,400.67538
4,TTPDVIFVFGFR,2,[402.79162614 427.7358606 ],NaN,TTPDVIFVFGFR,Not modified,True,0,421.81415,403.15720
...,...,...,...,...,...,...,...,...,...,...
876,NFASYGHLMGEMPR,3,[483.87045898 511.36608634],NaN,NFASYGHLMGEMPR,Not modified,True,0,511.56620,481.44970
877,AGDKDDITEPAVCALR,3,[507.79923073 551.58281351],NaN,AGDKDDITEPAVCALR,13|Carbamidomethyl,True,0,523.96110,490.60626
878,LASVPAGGAVAVSAAPGSAAPAAGSAPAAAEEK,2,[580.26127973 555.59131519],NaN,LASVPAGGAVAVSAAPGSAAPAAGSAPAAAEEK,Not modified,True,0,591.74500,530.70900
879,EEAAAVPAAAPDDLALLK,2,[454.76034697 468.57288974],NaN,EEAAAVPAAAPDDLALLK,Not modified,True,0,495.30800,463.77340
